In [ ]:
import os, glob
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import loadmat

# EEG (Electroencephalography) Sample Analysis

In [ ]:
eeg_data_dir = "data/samples/EEG"
eeg_sub_dirs = os.listdir(eeg_data_dir)
eeg_sub_dir_file_dict = dict()

for sub_dir in eeg_sub_dirs:
    eeg_sub_dir_file_dict[sub_dir] = glob.glob(os.path.join(eeg_data_dir, sub_dir, "**/*.mat"), recursive=True)

eeg_sub_dir_file_dict

In [ ]:
sample_eeg = eeg_sub_dir_file_dict["VP001-EEG"]
sample_eeg_cnt = loadmat(sample_eeg[0])["cnt_dsr"]
sample_eeg_mrk = loadmat(sample_eeg[6])["mrk_dsr"]

In [ ]:
print(f"EEG cnt keys: {sample_eeg_cnt.dtype}")
print(f"EEG mrk keys: {sample_eeg_mrk.dtype}")

In [ ]:
channel_eeg_labels = sample_eeg_cnt["clab"][0][0][0]                                    # channel labels
channel_eeg_labels = [ch.tolist()[0] for ch in channel_eeg_labels.tolist()]
cnt_eeg_sfreq = sample_eeg_cnt["fs"][0][0][0][0]                                        # sample frequency
cnt_eeg_signals = sample_eeg_cnt["x"][0][0]
cnt_eeg_event_onset = sample_eeg_mrk["time"][0][0].reshape(-1)                          # event / epoch onsets (millisecs)
cnt_eeg_event_classes = sample_eeg_mrk["event"][0][0][0][0][0].reshape(-1)              # classes
cnt_eeg_ohe_event_classes = sample_eeg_mrk["y"][0][0].T                                 # one hot encoded classes
cnt_eeg_class_names = sample_eeg_mrk["className"][0][0].reshape(-1)                     # class names
zeros = np.zeros_like(cnt_eeg_ohe_event_classes, dtype=int)
eeg_time_indices = ((cnt_eeg_event_onset / 1000) * cnt_eeg_sfreq).astype(int).tolist()  # time indices of each event onset
eeg_time_indices.append(cnt_eeg_signals.shape[0])                                       # to capture the last segment

In [ ]:
cnt_eeg_signals.shape, len(eeg_time_indices)

In [ ]:
eeg_start_times = eeg_time_indices[:-1]
eeg_end_times = eeg_time_indices[1:]

eeg_epochs = []
eeg_classes = []
for start, end, event_label in zip(eeg_start_times, eeg_end_times, cnt_eeg_event_classes):
    eeg_epochs.append(cnt_eeg_signals[start:end])
    eeg_classes.append(event_label)

print(f"Number of segments / epochs: {len(eeg_epochs)}")

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(25, 7))
sample_idx = [0, 20, 13, -1]

for i, idx in enumerate(sample_idx):
    sample_epoch = eeg_epochs[idx]
    sample_class = eeg_classes[idx]
    for t in sample_epoch.T:
        axs[i].plot(t)
        axs[i].set_title(f"class label: {sample_class}")
        axs[i].set_xlabel("timesteps")
        axs[i].set_ylabel("channels")

# NRIS (Near-Infrared Spectroscopy) Sample Analysis

In [ ]:
nirs_data_dir = "data/samples/NIRS"
nirs_sub_dirs = os.listdir(nirs_data_dir)
nirs_sub_dir_file_dict = dict()

for sub_dir in nirs_sub_dirs:
    nirs_sub_dir_file_dict[sub_dir] = glob.glob(os.path.join(nirs_data_dir, sub_dir, "**/*.mat"), recursive=True)

nirs_sub_dir_file_dict

In [ ]:
sample_nirs = nirs_sub_dir_file_dict["VP001-NIRS"]
sample_nirs_cnt = loadmat(sample_nirs[1])["cnt_nback"]
nsample_nirs_mrk = loadmat(sample_nirs[7])["mrk_nback"]

In [ ]:
print(f"NIRS cnt keys: {sample_nirs_cnt.dtype}")
print(f"NIRS mrk keys: {nsample_nirs_mrk.dtype}")

In [ ]:
nirs_cnt_oxy = sample_nirs_cnt['oxy'][0][0]
nirs_cnt_deoxy =sample_nirs_cnt ['deoxy'][0][0]

In [ ]:
print(f"NIRS cnt oxy keys: {nirs_cnt_oxy.dtype}")
print(f"NIRS cnt deoxy keys: {nirs_cnt_deoxy.dtype}")

### Oxygenated hemoglobin (oxy-Hb) data

In [ ]:
channel_nirs_labels = nirs_cnt_oxy["clab"][0][0][0]                                         # channel labels
channel_nirs_labels = [ch.tolist()[0] for ch in channel_nirs_labels.tolist()]
cnt_nirs_sfreq = nirs_cnt_oxy["fs"][0][0][0][0]                                             # sample frequency
cnt_nirs_signals = nirs_cnt_oxy["x"][0][0]
cnt_nirs_event_onset = nsample_nirs_mrk["time"][0][0].reshape(-1)                           # event / epoch onsets (millisecs)
cnt_nirs_event_classes = nsample_nirs_mrk["event"][0][0][0][0][0].reshape(-1)               # classes
cnt_nirs_ohe_event_classes = nsample_nirs_mrk["y"][0][0].T                                  # one hot encoded classes
cnt_nirs_class_names = nsample_nirs_mrk["className"][0][0].reshape(-1)                      # class names
zeros = np.zeros_like(cnt_nirs_ohe_event_classes, dtype=int)
nirs_time_indices = ((cnt_nirs_event_onset / 1000) * cnt_nirs_sfreq).astype(int).tolist()   # time indices of each event onset
nirs_time_indices.append(cnt_nirs_signals.shape[0])                                         # to capture the last segment

In [ ]:
cnt_eeg_signals.shape, len(nirs_time_indices), cnt_nirs_ohe_event_classes.shape

In [ ]:
cnt_nirs_event_classes, cnt_nirs_class_names

In [ ]:
nirs_start_times = nirs_time_indices[:-1]
nirs_end_times = nirs_time_indices[1:]

nirs_epochs = []
nirs_classes = []
for start, end, event_label in zip(nirs_start_times, nirs_end_times, cnt_nirs_event_classes):
    nirs_epochs.append(cnt_nirs_signals[start:end])
    nirs_classes.append(event_label)

print(f"Number of segments / epochs: {len(nirs_epochs)}")

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(25, 7))
sample_idx = [0, 5, 10, -1]

for i, idx in enumerate(sample_idx):
    sample_epoch = nirs_epochs[idx]
    sample_class = nirs_classes[idx]
    for t in sample_epoch.T:
        axs[i].plot(t)
        axs[i].set_title(f"class label: {sample_class}")
        axs[i].set_xlabel("timesteps")
        axs[i].set_ylabel("channels")

### Deoxygenated hemoglobin (deoxy-Hb) data

In [ ]:
channel_nirs_labels = nirs_cnt_deoxy["clab"][0][0][0]                                         # channel labels
channel_nirs_labels = [ch.tolist()[0] for ch in channel_nirs_labels.tolist()]
cnt_nirs_sfreq = nirs_cnt_deoxy["fs"][0][0][0][0]                                             # sample frequency
cnt_nirs_signals = nirs_cnt_deoxy["x"][0][0]
cnt_nirs_event_onset = nsample_nirs_mrk["time"][0][0].reshape(-1)                           # event / epoch onsets (millisecs)
cnt_nirs_event_classes = nsample_nirs_mrk["event"][0][0][0][0][0].reshape(-1)               # classes
cnt_nirs_ohe_event_classes = nsample_nirs_mrk["y"][0][0].T                                  # one hot encoded classes
cnt_nirs_class_names = nsample_nirs_mrk["className"][0][0].reshape(-1)                      # class names
zeros = np.zeros_like(cnt_nirs_ohe_event_classes, dtype=int)
nirs_time_indices = ((cnt_nirs_event_onset / 1000) * cnt_nirs_sfreq).astype(int).tolist()   # time indices of each event onset
nirs_time_indices.append(cnt_nirs_signals.shape[0])                                         # to capture the last segment

In [ ]:
cnt_eeg_signals.shape, len(nirs_time_indices), cnt_nirs_ohe_event_classes.shape

In [ ]:
cnt_nirs_event_classes, cnt_nirs_class_names

In [ ]:
nirs_start_times = nirs_time_indices[:-1]
nirs_end_times = nirs_time_indices[1:]

nirs_epochs = []
nirs_classes = []
for start, end, event_label in zip(nirs_start_times, nirs_end_times, cnt_nirs_event_classes):
    nirs_epochs.append(cnt_nirs_signals[start:end])
    nirs_classes.append(event_label)

print(f"Number of segments / epochs: {len(nirs_epochs)}")

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(25, 7))
sample_idx = [0, 5, 10, -1]

for i, idx in enumerate(sample_idx):
    sample_epoch = nirs_epochs[idx]
    sample_class = nirs_classes[idx]
    for t in sample_epoch.T:
        axs[i].plot(t)
        axs[i].set_title(f"class label: {sample_class}")
        axs[i].set_xlabel("timesteps")
        axs[i].set_ylabel("channels")